In [71]:
import re
import spacy
import pdfplumber
from sumy.parsers.plaintext import PlaintextParser
from sumy.nlp.tokenizers import Tokenizer
from sumy.summarizers.lsa import LsaSummarizer
from reportlab.platypus import SimpleDocTemplate, Paragraph, Spacer
from reportlab.lib.styles import getSampleStyleSheet
import google.generativeai as genai
import json


In [72]:
nlp = spacy.load("en_core_web_sm")

In [73]:
def load_api_keys(path="C:\\Users\\Vivek\\Desktop\\Programs\\Projects\\Legal-Document-Simplifier\\key.txt"):
    with open(path, "r") as f:
        return [line.strip() for line in f if line.strip()]

class GeminiKeyManager:
    def __init__(self, path="C:\\Users\\Vivek\\Desktop\\Programs\\Projects\\Legal-Document-Simplifier\\key.txt"):
        self.keys = load_api_keys(path)
        self.index = 0
        self.configure_client()

    def configure_client(self):
        genai.configure(api_key=self.keys[self.index])
        self.model = genai.GenerativeModel("gemini-1.5-flash")

    def rotate_key(self):
        self.index = (self.index + 1) % len(self.keys)
        self.configure_client()
        print(f"Switched to API key #{self.index+1}")

    def safe_generate(self, prompt):
        try:
            response = self.model.generate_content(prompt)
            return response
        except Exception as e:
            print(f"Error with key #{self.index+1}: {e}")
            self.rotate_key()
            # retry once with new key
            return self.model.generate_content(prompt)

gemini_keys = GeminiKeyManager()


In [74]:
def extract_text(pdf_file):
    text = ""
    with pdfplumber.open(pdf_file) as pdf:
        for page in pdf.pages:
            extracted = page.extract_text()
            if extracted:
                text += extracted + "\n"
    return text

In [75]:
def summarize_document(text, sentence_count=7):
    parser = PlaintextParser.from_string(text, Tokenizer("english"))
    summarizer = LsaSummarizer()
    summary_sentences = summarizer(parser.document, sentence_count)
    return "\n".join(str(s) for s in summary_sentences)

In [76]:
def extract_entities(text):
    doc = nlp(text)
    entities = {
        "Parties": [ent.text for ent in doc.ents if ent.label_ in ["ORG", "PERSON"]],
        "Dates": [ent.text for ent in doc.ents if ent.label_ == "DATE"],
        "Money": [ent.text for ent in doc.ents if ent.label_ == "MONEY"],
        "Obligations": re.findall(
            r"\b(shall|must|agree to|responsible for)\b.*?\.",
            text,
            flags=re.IGNORECASE,
        ),
    }
    return entities



In [77]:
def analyze_risks(text):
    risks = []
    risk_keywords = {
        "Late Payments / Financial Penalties": r"late fee|penalt(y|ies)|interest|default in payment|overdue|delayed payment",
        "Termination & Breach": r"termination|breach of contract|void|cancellation|material breach|contract violation",
        "Confidentiality & Disclosure": r"disclosure|confidential|nda|non-disclosure|trade secret",
        "Liability & Indemnification": r"liable|indemnif(y|ication)|responsibility|hold harmless|compensation liability",
        "Intellectual Property Risks": r"intellectual property|copyright|patent|trademark|IP rights|proprietary|design rights",
        "Dispute Resolution (Indian Context)": r"arbitration|jurisdiction|venue|dispute resolution|litigation|mediation|conciliation",
        "Damages & Remedies": r"damages|compensation|losses|remedies|consequential damages|punitive damages|specific performance",
        "Defamation / Reputation": r"defamation|reputation|libel|slander|character assassination",
        "Automatic Renewal": r"auto-?renewal|automatic renewal|renewal term|extension of contract",
        "Legal Fees & Costs": r"advocate fees|legal fees|court costs|litigation expenses",
        "Employment / Labour Risks": r"non-compete|non compete|non-solicit|non solicit|employee|employment|labour|industrial dispute",
        "Governing Law / Jurisdiction (India)": r"governing law|laws of India|Indian Penal Code|IPC|jurisdiction of.*India|court of.*India|Supreme Court|High Court|Arbitration and Conciliation Act",
    }
    for risk, pattern in risk_keywords.items():
        if re.search(pattern, text, flags=re.IGNORECASE):
            risks.append(risk)

    return risks if risks else ["No obvious risks detected"]


In [78]:
def enhance_with_gemini(summary, entities, risks):
    prompt = f"""
    You are a legal assistant.

    Here are extracted details from a legal document:

    --- Summary ---
    {summary}

    --- Entities ---
    {entities}

    --- Risks ---
    {risks}

    Task:
    1. Fix grammar and improve readability of the summary.
    2. Add a "suggestion" field (1–3 sentences of advice).
    3. Return a single JSON with keys:
       - summary
       - parties
       - date/time(
           mention context of a specific date/time with format "<b>context:</b> Value"
           example: 
                <b>Age of the cat:</b> 7 months
                <b>Date of the incident:</b> 2022-01-01
                <b>Document valid until:</b> 2023-01-01
            if any number is not a valid date with context unrelated to date/time, do not include it
            )
       - money/penalties
       - obligations
       - risks (string with multiple lines. Each risk must be formatted as:  
            "<b>• Risk Name</b>: Risk description(1line) asper Indian law" 
            with each risk on a new line, separated by <br/> inside the JSON string)
       - suggestion
    ONLY return valid JSON. Do not include triple backticks or any extra text.
    the entities (parties, date/time, money/penalties, obligations) as well as risks should return plain text, not inside [].
    if data is not available for any of the fields,return few words that might be relevant insted of none.
    """
    response = gemini_keys.safe_generate(prompt)
    result = response.text.strip()

    # Remove ```json ... ``` wrappers if present
    if result.startswith("```"):
        result = re.sub(r"^```(?:json)?", "", result.strip(), flags=re.IGNORECASE).strip()
        result = re.sub(r"```$", "", result).strip()

    return result

In [79]:
def save_json_to_pdf(json_str, filename="legal_analysis.pdf"):
    try:
        data = json.loads(json_str)
    except:
        print("Gemini response not valid JSON. Saving raw text.")
        data = {"raw_output": json_str}

    doc = SimpleDocTemplate(filename)
    styles = getSampleStyleSheet()
    flow = [Paragraph("AI Legal Document Analysis", styles["Title"]), Spacer(1, 20)]

    for key, value in data.items():
        flow.append(Paragraph(f"<b>{key.title()}:</b>", styles["Heading3"]))
        flow.append(Spacer(1, 6))
        flow.append(Paragraph(str(value), styles["Normal"]))
        flow.append(Spacer(1, 12))

    doc.build(flow)
    print(f"✅ Saved: {filename}")


In [80]:
def process_documents(pdf_files):
    for pdf_file in pdf_files:
        print(f"\nProcessing: {pdf_file}\n{'-'*50}")
        text = extract_text(pdf_file)

        # Offline processing
        summary = summarize_document(text)
        entities = extract_entities(text)
        risks = analyze_risks(text)

        print("Offline Summary/Entities/Risks ready.")

        # Gemini enhancement
        json_result = enhance_with_gemini(summary, entities, risks)
        print("Gemini JSON Result:\n", json_result[:400], "...\n")

        # Save PDF
        save_json_to_pdf(json_result, filename=f"{pdf_file[:-4]}_analysis.pdf")


In [81]:
pdf_files = [r"C:\Users\Vivek\Desktop\Programs\Projects\Legal-Document-Simplifier\silver_certf.pdf"]  # Replace with your PDFs
process_documents(pdf_files)


Processing: C:\Users\Vivek\Desktop\Programs\Projects\Legal-Document-Simplifier\silver_certf.pdf
--------------------------------------------------
Offline Summary/Entities/Risks ready.
Gemini JSON Result:
 {
  "summary": "This certificate confirms that Dr. Millon Rout (VCI Regd No. 10371) examined Silver, a seven-month-old female Persian cat owned by Mr. Amritansh Panigrahi of Bhubaneswar, Odisha (phone: 6371719825).  The examination revealed Silver to be free from infection or contagious disease and in apparent good health.  Silver is fully vaccinated for her age; vaccination records are attached.  ...

✅ Saved: C:\Users\Vivek\Desktop\Programs\Projects\Legal-Document-Simplifier\silver_certf_analysis.pdf
